Resources:
- https://quantum-computing.ibm.com/composer/files/new
- https://wybiral.github.io/quantum/
- https://jarrodmcclean.com/basic-quantum-circuit-simulation-in-python/
- https://quantumcomputing.stackexchange.com/questions/29454/why-do-quantum-computing-simulators-have-the-measurement-function

In [19]:
import numpy as np
import scipy as sp

In [20]:
# operator matrices

norm_matr = lambda state: state / sp.linalg.norm(state)

# identity
Id = np.array([
    [1,0],
    [0,1]
])
# pauli x
X = np.array([
    [0,1],
    [1,0]
])
# pauli y
Y = np.array([
    [0,0-1j],
    [0+1j,0]
])
# pauli z
Z = np.array([
    [1,0],
    [0,-1]
])
# hadamard
H = norm_matr(np.array([
    [1,1],
    [1,-1]
]))
# phase
S = np.array([
    [1,0],
    [0,0+1j]
])
# |0><0>
P_0 = np.array([
    [1,0],
    [0,0]
])
# |1><1>
P_1 = np.array([
    [0,0],
    [0,1]
])

In [21]:
class QState:
    def __init__(self, qbit_cnt, info=False):
        self.qbit_cnt = qbit_cnt
        # create state |00...0> with qbit_cnt amount of zeros
        self.state = np.zeros(2**self.qbit_cnt, dtype=complex)
        self.state[0] = 1

        if info:
            max_bin_width = len(bin(len(self.state) - 1)[2:])
            binary_str = bin(0)[2:].zfill(max_bin_width)
            print("Initial state: |{}>".format(binary_str))
    
    # show all states including the impossible one (unless
    # otherwise noted) and show according probabilities
    def show_state_and_probs(self, reduced=False):
        max_len = 0
        for element in self.state:
            if reduced and element == 0+0j:
                continue

            curr_len = max(len(str(element.real)), len(str(element.imag)))
            if curr_len > max_len:
                max_len = curr_len
        
        max_bin_width = len(bin(len(self.state) - 1)[2:])

        for i, element in enumerate(self.state):
            if reduced and element == 0+0j:
                continue

            real_part_str = "{:.{width}f}".format(element.real, width=max_len)
            imaginary_part_str = "{:.{width}f}".format(element.imag, width=max_len)
            formatted_element = "{}+{}i".format(real_part_str, imaginary_part_str)

            binary_str = bin(i)[2:].zfill(max_bin_width)

            prob_str = str(round((np.absolute(element)**2)*100, 4))

            if i > 0:
                print("+ {}|{}>\t-> {}%".format(formatted_element, binary_str, prob_str))
            else:    
                print("  {}|{}>\t-> {}%".format(formatted_element, binary_str, prob_str))
        
    ###
    ### GATE CREATION HELPER ###
    ###
    def op_mats_arr_to_tens(self, op_mats_arr):
        result = np.array([[1.0]])
        for op_mat in op_mats_arr:
            result = np.kron(result, op_mat)
        
        return result

    def sing_qbit_op(self, op_mat, pos):
        op_mats_arr = (
              (pos)*[Id]
            + (1)*[op_mat]
            + (self.qbit_cnt-pos-1)*[Id]
        )
        gate = self.op_mats_arr_to_tens(op_mats_arr)
        self.state = np.matmul(gate, self.state)
    
    def ctrl_qbit_op(self, ctrl_pos, targ_pos, targ_op_mat):
        # one of the cases will be zero, and the other one will
        # then represent the final gate

        # case 1
        op_mats_arr1 = (
              (ctrl_pos)*[Id]
            + (1)*[P_0]
            + (self.qbit_cnt-ctrl_pos-1)*[Id]
        )
        gate1 = self.op_mats_arr_to_tens(op_mats_arr1)
        
        # case 2
        # if ctrl_pos > targ_pos:
        #     op_mats_arr2 = (
        #         (ctrl_pos)*[Id]
        #         + (1)*[P_1]
        #         + (targ_pos-ctrl_pos-1)*[Id]
        #         + (1)*[targ_op_mat]
        #         + (self.qbit_cnt-targ_pos-1)*[Id]
        #     )
        # elif targ_pos > ctrl_pos:
        #     op_mats_arr2 = (
        #         (targ_pos)*[Id]
        #         + (1)*[targ_op_mat]
        #         + (ctrl_pos-targ_pos-1)*[Id]
        #         + (1)*[P_1]
        #         + (self.qbit_cnt-ctrl_pos-1)*[Id]
        #     )
        op_mats_arr2 = (
            (
                (ctrl_pos)*[Id]
                + (1)*[P_1]
                + (targ_pos-ctrl_pos-1)*[Id]
                + (1)*[targ_op_mat]
                + (self.qbit_cnt-targ_pos-1)*[Id]
            ) if ctrl_pos > targ_pos else
            (
                (targ_pos)*[Id]
                + (1)*[targ_op_mat]
                + (ctrl_pos-targ_pos-1)*[Id]
                + (1)*[P_1]
                + (self.qbit_cnt-ctrl_pos-1)*[Id]
            )
        )
        gate2 = self.op_mats_arr_to_tens(op_mats_arr2)

        gate = gate1 + gate2
        self.state = np.matmul(gate, self.state)

    def up_down_ctrl_qbit_op(self):
        pass

    ###
    ### SINGLE-QUBIT GATES ###
    ###

    def pauli_x(self, pos):
        self.sing_qbit_op(X, pos)

    def pauli_y(self, pos):
        self.sing_qbit_op(Y, pos)

    def pauli_z(self, pos):
        self.sing_qbit_op(Z, pos)

    def hadamard(self, pos):
        self.sing_qbit_op(H, pos)

    ###
    ### MULTI-QUBIT GATES ###
    ###
    
    # strictly only if control bit above target bit
    def cnot(self, ctrl_pos, targ_pos):
        self.ctrl_qbit_op(ctrl_pos, targ_pos, X)

    def up_down_cnot(self, targ_pos, ctrl_pos):
        pass

    def cz(self, ctrl_pos, targ_pos):
        self.ctrl_qbit_op(ctrl_pos, targ_pos, Z)

    def up_down_cz(self):
        # does this even exist
        pass
    
    def cp(self, ctrl_pos, targ_pos):
        self.ctrl_qbit_op(ctrl_pos, targ_pos, S)
    
    def up_down_cp(self, targ_pos, ctrl_pos):
        pass



In [22]:
### Testing

qs = QState(4, True)

qs.show_state_and_probs(True)

Initial state: |0000>
  1.000+0.000i|0000>	-> 100.0%


In [23]:
# some tests

Id = "hi"
X = "bye"

arr = 4*[Id] + 2*[X]
print(arr)

['hi', 'hi', 'hi', 'hi', 'bye', 'bye']
